In [134]:
from pathlib import Path
import pandas as pd

In [135]:
def grouped_results(merged_df, agg, target_column, join_symbol="±"):
    formating = lambda x: "{:.2f}".format(x)
    grouped_mean_formatted = merged_df.groupby(agg)[target_column].mean().map(formating)
    grouped_std_formatted = merged_df.groupby(agg)[target_column].std().map(formating)
    result_df = pd.concat([grouped_mean_formatted, grouped_std_formatted], axis=1)
    result_df.columns = ["Mean", "Std"]

    # Combine mean and std columns to display mean ± std
    result_df["Mean ± Std"] = result_df["Mean"] + f"{join_symbol}" + result_df["Std"]

    result_df["Mean ± Std"] = result_df["Mean ± Std"].map(lambda x: f"${x}$")
    return result_df


def compute_diffs(merged_df):
    reg_false_df = merged_df[merged_df["reg_layers"] == False]
    reg_true_df = merged_df[merged_df["reg_layers"] == True]

    # Merge the two dataframes based on 'num_classes', 'approach', 'dataset', and 'seed'
    merged_reg_diff = pd.merge(
        reg_false_df,
        reg_true_df,
        on=["num_tasks", "approach", "dataset", "seed"],
        suffixes=("_false", "_true"),
        how="inner",
    )
    merged_reg_diff["avg_acc_tag_diff"] = (
        merged_reg_diff["avg_acc_tag_true"] - merged_reg_diff["avg_acc_tag_false"]
    )
    return merged_reg_diff


def get_merged_df(paths):
    dfs = [pd.read_csv(path) for path in paths]

    merged_df = pd.concat(dfs, axis=0)
    return merged_df

In [141]:
root = Path("csvs")
convnext = root / "convnext_tiny"
paths = list(convnext.glob("cifar*.csv"))


merged_df = get_merged_df(paths)

In [142]:
agg = ["num_tasks", "dataset", "approach", "reg_layers"]
grouped_results(merged_df, agg, "avg_acc_tag", join_symbol="\pm")

Mean   Std      Mean ± Std
num_tasks dataset        approach   reg_layers                             
5         cifar100_fixed ewc        False       29.57  0.03  $29.57\pm0.03$
                                    True        33.82  0.32  $33.82\pm0.32$
                         finetuning False       26.21  0.85  $26.21\pm0.85$
                                    True        30.71  0.68  $30.71\pm0.68$
                         lwf        False       40.70  0.12  $40.70\pm0.12$
                                    True        49.38  0.43  $49.38\pm0.43$
                         replay     False       34.53  0.30  $34.53\pm0.30$
                                    True        37.30  1.14  $37.30\pm1.14$
10        cifar100_fixed ewc        False       16.92  0.67  $16.92\pm0.67$
                                    True        24.11  0.42  $24.11\pm0.42$
                         finetuning False       15.31  0.45  $15.31\pm0.45$
                                    True        20.46  0.12  $20.46\pm0.12$
                         lwf        False       29.83  0.25  $29.83\pm0.25$
                                    True        39.69  0.55  $39.69\pm0.55$
                         replay     False       28.59  0.16  $28.59\pm0.16$
                                    True        32.05  0.85  $32.05\pm0.85$
20        cifar100_fixed ewc        False        8.87  0.65   $8.87\pm0.65$
                                    True        15.85  0.65  $15.85\pm0.65$
                         finetuning False        8.19  0.33   $8.19\pm0.33$
                                    True        13.68  0.45  $13.68\pm0.45$
                         lwf        False       23.01  1.07  $23.01\pm1.07$
                                    True        31.53  0.91  $31.53\pm0.91$
                         replay     False       25.42  0.31  $25.42\pm0.31$
                                    True        30.22  0.09  $30.22\pm0.09$

In [143]:
agg = ["num_tasks", "dataset", "approach"]
merged_reg_diff = compute_diffs(merged_df)
grouped_results(merged_reg_diff, agg, "avg_acc_tag_diff", join_symbol="\pm")

Mean   Std     Mean ± Std
num_tasks dataset        approach                             
5         cifar100_fixed ewc         4.25  0.29  $4.25\pm0.29$
                         finetuning  4.50  0.17  $4.50\pm0.17$
                         lwf         8.69  0.54  $8.69\pm0.54$
                         replay      2.77  1.44  $2.77\pm1.44$
10        cifar100_fixed ewc         7.20  0.25  $7.20\pm0.25$
                         finetuning  5.15  0.57  $5.15\pm0.57$
                         lwf         9.86  0.30  $9.86\pm0.30$
                         replay      3.46  0.70  $3.46\pm0.70$
20        cifar100_fixed ewc         6.97  0.01  $6.97\pm0.01$
                         finetuning  5.49  0.79  $5.49\pm0.79$
                         lwf         8.52  0.16  $8.52\pm0.16$
                         replay      4.80  0.40  $4.80\pm0.40$